In [ ]:
!pip -q install faiss-cpu peft accelerate bitsandbytes sentencepiece evaluate rouge-score nltk bert_score textblob

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
import os, gc, re, numpy as np, pandas as pd, torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from textblob import TextBlob

import faiss
from sentence_transformers import SentenceTransformer

from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration,
    get_linear_schedule_with_warmup
)
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW


In [ ]:
# Load
df = pd.read_csv('temp_clean.csv')
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'], errors='ignore')
df = df.drop(columns=['cleaned_review','cleaned_reply'], errors='ignore')

# (Optional) Dedup like you did earlier
dedup_cols = ['placeInfo/id','reply','placeInfo/name','placeInfo/rating','title','rating','user/userLocation/shortName']
df = df.drop_duplicates(subset=[c for c in dedup_cols if c in df.columns], keep='first').reset_index(drop=True)

# Basic length guards to avoid extreme outliers
df['reply_length']  = df['cleaned_response2'].astype(str).str.len()
df = df[df['reply_length'] <= 1200].reset_index(drop=True)

# Sentiment route for 3-star via polarity (same heuristic)
def polarity(x):
    try: return TextBlob(str(x)).sentiment.polarity
    except: return 0.0
df['review_sentiment'] = df['cleaned_review2'].apply(polarity)

# Build a single control label for sentiment
def label_sentiment(row):
    if row['rating'] >= 4: return 'positive'
    if row['rating'] <= 2: return 'negative'
    return 'positive' if row['review_sentiment'] >= 0 else 'negative'
df['sent_label'] = df.apply(label_sentiment, axis=1)

# Optional: star control tag (helps the single model learn tone)
def star_tag(r):
    r = int(r) if pd.notnull(r) else 3
    r = min(max(r,1),5)
    return f"<STAR_{r}>"
df['star_tag'] = df['rating'].apply(star_tag)

# Split once for the single generator
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['sent_label'])
val_df,   test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['sent_label'])

print("Train/Val/Test:", train_df.shape, val_df.shape, test_df.shape)
print(train_df['sent_label'].value_counts(normalize=True).round(3))


Train/Val/Test: (5884, 14) (736, 14) (736, 14)
sent_label
positive    0.907
negative    0.093
Name: proportion, dtype: float64


In [ ]:
class ReviewRetrieval:
    def __init__(self, df, model_name="sentence-transformers/all-mpnet-base-v2",
                 embed_col="cleaned_review2", reply_col="cleaned_response2", use_gpu=True):
        self.df = df.reset_index(drop=True)
        self.embed_col = embed_col
        self.reply_col = reply_col

        print(f"Loading SBERT: {model_name}")
        self.model = SentenceTransformer(model_name)
        if use_gpu:
            try:
                self.model = self.model.to('cuda')
                print("SBERT on GPU")
            except:
                print("SBERT on CPU")

        self._build_index()

    def _build_index(self):
        reviews = self.df[self.embed_col].fillna("").tolist()
        embs = self.model.encode(
            reviews, batch_size=32, convert_to_numpy=True,
            normalize_embeddings=True, show_progress_bar=True
        ).astype("float32")
        self.embeddings = embs
        dim = embs.shape[1]
        self.index = faiss.IndexFlatIP(dim)
        self.index.add(embs)
        print("FAISS index size:", self.index.ntotal)

    def retrieve(self, query_text, top_k=3):
        if not isinstance(query_text, str) or not query_text.strip():
            return []
        q = self.model.encode([query_text], convert_to_numpy=True, normalize_embeddings=True).astype("float32")
        D, I = self.index.search(q, top_k)
        return self.df.iloc[I[0]][self.reply_col].tolist()

# Build on TRAIN ONLY (avoid leakage)
retriever = ReviewRetrieval(train_df)


Loading SBERT: sentence-transformers/all-mpnet-base-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SBERT on GPU


Batches:   0%|          | 0/184 [00:00<?, ?it/s]

FAISS index size: 5884


In [ ]:
def build_prompt(review_text, retrieved_replies, sent_label, star_tag):
    sent_tag = "<POSITIVE>" if sent_label == "positive" else "<NEGATIVE>"
    retrieved_block = "".join([f"[EXAMPLE_{i+1}] {r}\n" for i, r in enumerate(retrieved_replies)])

    prompt = f"""Instruction:
Generate a professional, context-aware manager reply to the guest review.

Sentiment: {sent_tag}
RatingTag: {star_tag}

Review:
{review_text}

Relevant Past Replies:
{retrieved_block}

Manager Reply:"""
    return prompt.strip()


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
# add our control tokens to tokenizer vocab (so they aren't split oddly)
special_tokens = {"additional_special_tokens": ["<POSITIVE>","<NEGATIVE>","<STAR_1>","<STAR_2>","<STAR_3>","<STAR_4>","<STAR_5>"]}
tokenizer.add_special_tokens(special_tokens)

class GenDataset(Dataset):
    def __init__(self, df, retriever, tokenizer, max_in_len=512, max_out_len=200):
        self.df = df.reset_index(drop=True)
        self.retriever = retriever
        self.tok = tokenizer
        self.max_in_len = max_in_len
        self.max_out_len = max_out_len

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        review = str(row["cleaned_review2"])
        reply  = str(row["cleaned_response2"])
        sent_label = row["sent_label"]
        star_tag   = row["star_tag"]

        retrieved = self.retriever.retrieve(review, top_k=3)
        prompt = build_prompt(review, retrieved, sent_label, star_tag)

        enc = self.tok(prompt, max_length=self.max_in_len, truncation=True, padding="max_length", return_tensors="pt")
        dec = self.tok(reply,  max_length=self.max_out_len, truncation=True, padding="max_length", return_tensors="pt")

        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "labels": dec["input_ids"].squeeze()
        }

BATCH_SIZE=4; MAX_IN=512; MAX_OUT=200

train_ds = GenDataset(train_df, retriever, tokenizer, MAX_IN, MAX_OUT)
val_ds   = GenDataset(val_df,   retriever, tokenizer, MAX_IN, MAX_OUT)
test_ds  = GenDataset(test_df,  retriever, tokenizer, MAX_IN, MAX_OUT)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
# resize token embeddings to account for new special tokens
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

def train(model, train_loader, val_loader, epochs=3, lr=3e-5):
    optimizer = AdamW(model.parameters(), lr=lr)
    total_steps = len(train_loader)*epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(0.1*total_steps), num_training_steps=total_steps
    )
    for ep in range(1, epochs+1):
        model.train(); tr_loss=0
        pbar=tqdm(train_loader, desc=f"Epoch {ep}/{epochs}")
        for batch in pbar:
            batch = {k:v.to(device) for k,v in batch.items()}
            out = model(**batch)
            loss = out.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step(); scheduler.step(); optimizer.zero_grad()
            tr_loss += loss.item(); pbar.set_postfix(loss=f"{loss.item():.4f}")
        print(f"Train loss: {tr_loss/len(train_loader):.4f}")

        model.eval(); val_loss=0
        with torch.no_grad():
            for batch in val_loader:
                batch = {k:v.to(device) for k,v in batch.items()}
                out = model(**batch); val_loss += out.loss.item()
        print(f"Val loss: {val_loss/len(val_loader):.4f}")
    return model

model = train(model, train_loader, val_loader, epochs=3, lr=3e-5)
model.save_pretrained("single_generator_flan_t5_base")
tokenizer.save_pretrained("single_generator_flan_t5_base")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Epoch 1/3: 100%|██████████| 1471/1471 [23:03<00:00,  1.06it/s, loss=0.0027]


Train loss: 1.8637
Val loss: 1.5671


Epoch 2/3: 100%|██████████| 1471/1471 [22:51<00:00,  1.07it/s, loss=0.0002]


Train loss: 0.0338
Val loss: 1.4778


Epoch 3/3: 100%|██████████| 1471/1471 [22:50<00:00,  1.07it/s, loss=0.4988]


Train loss: 0.0295
Val loss: 1.4957


('single_generator_flan_t5_base/tokenizer_config.json',
 'single_generator_flan_t5_base/special_tokens_map.json',
 'single_generator_flan_t5_base/spiece.model',
 'single_generator_flan_t5_base/added_tokens.json',
 'single_generator_flan_t5_base/tokenizer.json')

In [ ]:
model_path = "/content/drive/MyDrive/single_generator_flan_t5_base"
tokenizer_path = "/content/drive/MyDrive/single_generator_flan_t5_base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
model = model.to(device)

In [ ]:
def generate_reply_single(review_text, rating=None, sentiment="auto", top_k=3, max_new_tokens=200):
    # sentiment
    if sentiment == "auto":
        sent_label = "positive" if TextBlob(str(review_text)).sentiment.polarity > 0 else "negative"
    else:
        sent_label = sentiment.lower()

    # star tag
    star = f"<STAR_{min(max(int(rating) if rating is not None else 3,1),5)}>"
    retrieved = retriever.retrieve(review_text, top_k=top_k)
    prompt = build_prompt(review_text, retrieved, sent_label, star)

    enc = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        out = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            num_beams=5,
            no_repeat_ngram_size=3,
            early_stopping=True
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)


Dear Guest, We are truly grateful for your wonderful feedback and delighted to hear that your stay exceeded your expectations. We appreciate your loyalty and look forward to welcoming you back on your next visit. Your kind words are a great encouragement for us. PERSON NAME , HOTEL_NAME> Manager PERSON_N
Dear Guest, Greetings from HOTEL_NAME>! Thank you for taking out time to share your valuable review. Please accept our sincere apologies for the inconvenience you faced during your stay with us. We have noted down the several areas of improvement highlighted by you and have shared them with my team. Your feedback goes a long way in helping us improve our existing services and quality standards. I hope you will reconsider and give us another chance to host you again. Best Wishes, PERSON NAME LOCATION>


In [ ]:
print(generate_reply_single('''i stayed with my wife one night its was amazing experience great hospitality room boy and front office guys highly recommended thank you all of you.

Room rent is affordable house keeping well maintained the room very comfortable easy easy to travel anywhere from the hotel'''))

Dear Guest, Thanks for choosing and staying with us during your recent visit to LOCATION>. We also appreciate your effort towards sharing your feedback on public platform which is always beneficial for other guest to find their stay. Looking forward to serve you again in near future. Regards, PERSON NAME General Manager


In [ ]:
print(generate_reply_single('''Was there beginning of the month, a moderate 3star budget hotel with basic facilities. The service staffs are from various countries but friendly even though there are not well versed with English. They tried to assist as much as they could immediately and promptly. Overall premises and rooms were kept clean even though minor things can be found. However, the facilities wise on obtaining iron, hairdryer, mineral water, extra towels or amenities was challenging. On last day of the departure met Mr Thiagaraj the hotel manager; he was friendly and willing to listen on The feedback. Kudos to Mathesh, his F&B team & FO team Shaam, Don & Sulaiman.
They are doing great job and keep it up. Improve on the minor issues and other than that it’s a great hotel to stay.'''))

Dear Guest, Greetings from HOTEL_NAME> LOCATION>!! I am delighted that you had a memorable stay with us, and that we were able to exceed your expectations. It is a pleasure to read about your flawless experience with our services and facilities. At ... ... we are committed to consistently providing warm, efficient, and discreet Service to our guests and their appreciation and acknowledgement are always our greatest rewards. Thank you so much for appreciating our team. I have shared your feedback with our concerned staff s. your appreciation means the world to our people and will inspire and encourage them in their work. continuous pursuit of excellence. We look forward to welcoming you back for many more enriching moments at ...


In [ ]:
print(generate_reply_single('''This is no where 3 star hotel, this seems a dharamshala
No Room service provided, even intercom was not there in room
Restaurant is closed
Staff is not helpful and for even basic needs like water, towels you need to come downstairs and ask people for help
'''))

Dear Guest, What an immense pleasure to read your review ! Thank you so much for sharing your feedback and comments of HOTEL_NAME> PERSON NAME ! I am glad to know that you enjoyed your stay and experienced a hospitable experience. Please stay with us whenever next you plan to visit PERONE NAME city. Thank you


In [ ]:
print(generate_reply_single('''My room was allotted on the fifth floor when I checked in at 11pm with my family. The health faucet was broken, then another room was changed where Air conditioning system was not working, then again changed room to a suite room where the tap of the wash basin was not fixed and had time to open and shut the tap as it was moving in all directions.
Would advise visitors to exercise caution.
I got a refund when I threatened to post my review in the local vernacular daily and Make My trip.com'''))

Dear Guest, Thank you very much for choosing to stay with us and sharing your feedback. Please accept my sincere apologies for the experience you had during your stay with we. We are looking into the matter. Your valuable feedback will guide us in the right direction to be able to provide a truly delightful experience to our guests. I am disappointed that we fell short this time, but I hope you will give us further opportunities to serve you better. Best Regards, PERSON NAME , HOTEL_NAME>


In [ ]:
import evaluate, nltk, numpy as np
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
rouge = evaluate.load("rouge"); bertscore = evaluate.load("bertscore")

def eval_split(df_eval, num_samples=50):
    refs, preds = [], []
    subset = df_eval.sample(n=min(num_samples, len(df_eval)), random_state=42).reset_index(drop=True)
    for i in tqdm(range(len(subset)), desc="Evaluating"):
        review = subset.loc[i, "cleaned_review2"]
        ref    = subset.loc[i, "cleaned_response2"]
        rating = subset.loc[i, "rating"] if "rating" in subset.columns else None
        sent_label = subset.loc[i, "sent_label"]
        gen = generate_reply_single(review, rating=rating, sentiment=sent_label)
        refs.append(str(ref)); preds.append(str(gen))

    r = rouge.compute(predictions=preds, references=refs)
    b = bertscore.compute(predictions=preds, references=refs, lang="en")

    smoothie = SmoothingFunction().method4
    bleu_vals = [sentence_bleu([r_.split()], p_.split(), smoothing_function=smoothie) for r_, p_ in zip(refs, preds)]
    return {
        "ROUGE-L": float(r["rougeL"]),
        "BLEU": float(np.mean(bleu_vals)),
        "BERTScore": float(np.mean(b["f1"])),
        "AvgGenLength": float(np.mean([len(p.split()) for p in preds]))
    }

print("Overall (test set):")
overall_scores = eval_split(test_df, num_samples=50); print(overall_scores)

print("\nPositive subset (test):")
pos_scores = eval_split(test_df[test_df['sent_label']=="positive"], num_samples=50); print(pos_scores)

print("\nNegative subset (test):")
neg_scores = eval_split(test_df[test_df['sent_label']=="negative"], num_samples=50); print(neg_scores)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Overall (test set):


Evaluating: 100%|██████████| 50/50 [02:38<00:00,  3.18s/it]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

{'ROUGE-L': 0.29650391702953616, 'BLEU': 0.09422266789915427, 'BERTScore': 0.8763648891448974, 'AvgGenLength': 71.46}

Positive subset (test):


Evaluating: 100%|██████████| 50/50 [02:15<00:00,  2.70s/it]


{'ROUGE-L': 0.28441540888598016, 'BLEU': 0.09417974378270064, 'BERTScore': 0.8684249615669251, 'AvgGenLength': 67.26}

Negative subset (test):


Evaluating: 100%|██████████| 50/50 [02:08<00:00,  2.56s/it]


{'ROUGE-L': 0.2363039488035276, 'BLEU': 0.04986658581281635, 'BERTScore': 0.8658000981807709, 'AvgGenLength': 71.28}


In [ ]:
!cp -r /content/single_generator_flan_t5_base/ "/content/drive/My Drive/"